In [10]:
# !pip install torch
# !pip install pandas
# !pip3 install pickle5

In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/Barley/single_env_barley/
# !pwd

In [12]:
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pickle
from scipy.stats import pearsonr
from sklearn.preprocessing import Normalizer

In [13]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [14]:
ATTENTION_HEAD = 2 
EMBID_DIM= 6
NLAYERS = 2
FEEDFORWARD_DIM = 256

In [15]:
np.random.seed(100)
torch.manual_seed(100)
NUM_FEATURES = 24866
KFOLD = 5
MUTUAL_INFO_THRESH = 0.05
FOLD = 4
MAX_EPOCH = 1000

torch.cuda.empty_cache()
torch.cuda.empty_cache()

In [16]:
training_file = '../Dataset/tr_hw_fold_{}.pkl'
test_file = '../Dataset/test_hw_fold_{}.pkl'
feature_file = '../Dataset/hw_selected_features.pkl'
validation_file = '../Dataset/validation_hw.pkl'

In [17]:
class BarleyDataset(Dataset):
    def __init__(self, file_path, feature_file):
        with open(file_path, 'rb') as pfile:
            self.data = pickle.load(pfile)

        with open(feature_file, 'rb') as pfile:
            self.selected_features = pickle.load(pfile)
           
        self.labels = self.data.iloc[:, -2:].to_numpy()
        self.data = self.data.iloc[:,:-2].to_numpy()
        self.transform()
        


    def transform(self):
        d = []
        for row in self.data:
            row = np.array(row)
            d.append(row)
        self.data = np.array(d)

        self.data = self.data[:,self.selected_features]

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, ind):
        return self.data[ind], self.labels[ind]

In [18]:
class TransformerModel(nn.Module):
    def __init__(self, attention_head, num_features, embed_dim, output_dim = 2, feedforward_dim = 256, dropout = 0.1, nlayers= 2):
        super(TransformerModel, self).__init__()
        self.num_features = num_features
        self.embed_dim = embed_dim
        
        self.embedding = nn.Linear(num_features, num_features * embed_dim)
        encoder_layers = nn.TransformerEncoderLayer(embed_dim, attention_head, feedforward_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        # self.fc_random = nn.Linear(num_markers * num_features, 768)
        self.dropout = nn.Dropout(p = 0.5)
        self.fc = nn.Linear(num_features * embed_dim, output_dim)
        self.relu = nn.ReLU()

        
    def forward(self, x):
        # x = x.view(x.shape[0], -1)
        # x = self.fc_random(x)
        x = self.embedding(x).view(x.shape[0], -1, self.embed_dim)
        x = self.transformer_encoder(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)

        x = self.fc(x)
        x = self.relu(x)
        return x

In [19]:
training_file = training_file.format(FOLD)
tr_barleyDataset = BarleyDataset(training_file, feature_file) 



In [20]:
print(len(np.unique(tr_barleyDataset.data)))

712


In [21]:
tr_loader = DataLoader(tr_barleyDataset, batch_size = 8, shuffle = True)

In [22]:
val_dataset = BarleyDataset(validation_file, feature_file)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
print(val_dataset.data.shape)

(80, 9833)


In [23]:
print(len(np.unique(val_dataset.data)))

681


### Training Model

In [24]:
# x = torch.rand(16, 1000, 1)
# print(x)
# model = TransformerModel(1, 1000, 1)

model = TransformerModel(attention_head = ATTENTION_HEAD, num_features = tr_barleyDataset.data.shape[1], 
                         embed_dim= EMBID_DIM, nlayers = NLAYERS, feedforward_dim= FEEDFORWARD_DIM)

In [25]:


device = 'cpu'

if torch.cuda.is_available():
    device = 'cuda:0'
    
model = model.to(device)

criterion = nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
best_avg_pcc = -1
best_fhb_pcc = 0
best_don_pcc = 0

best_model = model.state_dict()

no_improve = 0
best_epoch = 0
for epoch in range(MAX_EPOCH):
    tr_running_loss = 0
    count = 0
    pcc_fhb = 0
    pcc_don = 0
    model.train()
    for genos, phenos in tr_loader:
        #print(genos)
        genos = genos.to(device)
        genos = genos.float()
        
        phenos = phenos.to(device)
        phenos = phenos.float()
        
        
        optimizer.zero_grad()
        outputs = model(genos)

       
        #boxes = torch.FloatTe(boxes)
        loss = criterion(outputs, phenos)

        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        tr_running_loss += loss.item()
        pcc_fhb += pearsonr(phenos[:, 0].detach().cpu().numpy(), outputs[:, 0].detach().cpu().numpy())[0]
        pcc_don += pearsonr(phenos[:, 1].detach().cpu().numpy(), outputs[:, 1].detach().cpu().numpy() )[0]
        
        count+=1
    
    tr_avg_pcc = ((pcc_fhb + pcc_don) / 2) / count

    print('Epoch: {}'.format(epoch))
    print('TRAIN LOSS: {}, AVG PCC: {}, FHB PCC: {}, DON PCC: {}'.format(
        tr_running_loss / len(tr_loader), tr_avg_pcc, pcc_fhb / count, 
        pcc_don / count))
    print('********')


    count = 0
    pcc_fhb = 0
    pcc_don = 0
    validation_loss = 0

    model.eval()
    with torch.no_grad():
        for genos, phenos in val_loader:
            genos = genos.to(device)
            genos = genos.float()
            
            phenos = phenos.to(device)
            phenos = phenos.float()

            outputs = model(genos)
            
            loss = criterion(outputs, phenos)
            validation_loss += loss.item()

            pcc_fhb += pearsonr(phenos[:, 0].detach().cpu().numpy(), outputs[:, 0].detach().cpu().numpy())[0]
            pcc_don += pearsonr(phenos[:, 1].detach().cpu().numpy(), outputs[:, 1].detach().cpu().numpy() )[0]
            count+=1


    val_avg_pcc = ((pcc_fhb + pcc_don) / 2) / count
    
    print('VAL LOSS: {}, AVG PCC: {} FHB PCC: {}, DON PCC: {}'.format( 
          validation_loss / len(val_loader), val_avg_pcc, pcc_fhb / count, 
          pcc_don / count))
    
    if best_avg_pcc < val_avg_pcc:
        best_avg_pcc = val_avg_pcc
        best_fhb_pcc = pcc_fhb / count
        best_don_pcc = pcc_don / count
        best_model = model.state_dict()
        no_improve = 0
        best_epoch = epoch
    else:
        no_improve += 1

    print('********')
    print('BEST EPOCH: {} AVG PCC: {}, FHB PCC: {}, DON PCC: {}'.format(
        best_epoch, best_avg_pcc, best_fhb_pcc, best_don_pcc))
    
    print('===============================================================================')
    print()
    print()
    if no_improve >= 20:
        break

        
        

RuntimeError: CUDA out of memory. Tried to allocate 2.16 GiB (GPU 0; 7.92 GiB total capacity; 6.49 GiB already allocated; 433.62 MiB free; 6.49 GiB reserved in total by PyTorch)

In [ ]:
torch.save(best_model, '../outputs/model_' + str(FOLD) + '.pt')

### TEST MODEL

In [ ]:
test_dataset = BarleyDataset(test_file.format(FOLD), feature_file)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)
print(test_dataset.data.shape)

In [ ]:
model = TransformerModel(attention_head = ATTENTION_HEAD, num_features = tr_barleyDataset.data.shape[1], 
                         embed_dim= EMBID_DIM, nlayers = NLAYERS, feedforward_dim= FEEDFORWARD_DIM)

In [ ]:
model.load_state_dict(torch.load('../outputs/model_' + str(FOLD) + '.pt', map_location=device))
model = model.to(device)

In [ ]:
count = 0
pcc_fhb = 0
pcc_don = 0
test_loss = 0

model.eval()
true_fhb = []
true_don = []
predicted_fhb = []
predicted_don = []
test_loss = 0
with torch.no_grad():
    for genos, phenos in test_loader:
        genos = genos.to(device)
        genos = genos.float()
        
        phenos = phenos.to(device)
        phenos = phenos.float()

        outputs = model(genos)
        
        loss = criterion(outputs, phenos)
        test_loss += loss.item()

        pcc_fhb += pearsonr(phenos[:, 0].detach().cpu().numpy(), outputs[:, 0].detach().cpu().numpy())[0]
        pcc_don += pearsonr(phenos[:, 1].detach().cpu().numpy(), outputs[:, 1].detach().cpu().numpy() )[0]

        predicted_fhb += outputs[:, 0].detach().cpu().numpy().tolist()
        predicted_don += outputs[:, 1].detach().cpu().numpy().tolist()

        true_fhb += phenos[:, 0].detach().cpu().numpy().tolist()
        true_don += phenos[:, 1].detach().cpu().numpy().tolist()
        count+=1

    test_avg_pcc = ((pcc_fhb + pcc_don) / 2) / count
    
    print('TEST LOSS: {}, AVG PCC: {} FHB PCC: {}, DON PCC: {}'.format( 
          test_loss / len(test_loader), test_avg_pcc, pcc_fhb / count, 
          pcc_don / count))

In [ ]:
df = pd.DataFrame({'True FHB': true_fhb,
                   'Predicted FHB': predicted_fhb,
                   'True DON': true_don,
                   'Predicted DON': predicted_don})

In [ ]:
import plotly.express as px


fig = px.scatter(df, x="True FHB", y="Predicted FHB")
fig.show()

In [ ]:
fig = px.scatter(df, x="True DON", y="Predicted DON")
fig.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.clf()
plt.scatter(x= df['True FHB'].to_numpy(), y=df['Predicted FHB'].to_numpy())
plt.xlabel('True FHB')
plt.ylabel('Predicted FHB')
plt.show()
plt.savefig('../outputs/fhb_' + str(FOLD) + '.png')

In [ ]:
plt.clf()
plt.scatter(x= df['True DON'].to_numpy(), y=df['Predicted DON'].to_numpy())
plt.xlabel('True DON')
plt.ylabel('Predicted DON')
plt.show()
plt.savefig('../outputs/don_' + str(FOLD) + '.png')

In [ ]:
with open('../outputs/fhb_'+ str(FOLD) + '.pkl', 'wb') as file:
    pickle.dump(df[['True FHB', 'Predicted FHB']], file)

In [ ]:
with open('../outputs/don_'+ str(FOLD) + '.pkl', 'wb') as file:
    pickle.dump(df[['True DON', 'Predicted DON']], file)